In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from patsy import dmatrices

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [2]:
# pbp_data = pd.read_csv("./results/base-epa.csv", encoding="utf-8")
pbp_data = pd.DataFrame()
years = range(2014,2020)
for i in years:
    url = f"https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_{i}.parquet"
    print(f"Pulling url {url}")
    i_data = pd.read_parquet(url, engine='pyarrow')

    #sort=True eliminates a warning and alphabetically sorts columns
    pbp_data = pbp_data.append(i_data, sort=True)

#Give each row a unique index
pbp_data.reset_index(drop=True, inplace=True)
pbp_data.head()

Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2014.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2015.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2016.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2017.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2018.parquet
Pulling url https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2019.parquet


,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [3]:
game_data = pd.DataFrame()
spread_data = pd.DataFrame()

# downloaded files from collegefootballdata.com
def retrieveCfbDataFile(endpoint, year):
    return pd.read_csv(f"data/{endpoint}/{year}.csv", encoding='latin-1')

for i in range(2014, 2020):
    gm = retrieveCfbDataFile('games',i)
    gm['game_id'] = gm['id']
    gm['year'] = i
    gm['winner'] = np.where(gm.home_points > gm.away_points, gm.home_team, gm.away_team)
    game_data = game_data.append(gm, sort=False)
    
    bets = retrieveCfbDataFile('lines', i)
    bets = bets[bets.lineProvider == "consensus"]
    bets['game_id'] = bets['id']
    bets['year'] = i
    bets['spread'] = -1 * bets.spread # spread is negative for home favorite; positive for away favorite, but NFLfastR uses "(how many points home team was favored by) for this"
    spread_data = spread_data.append(bets, sort=False)
    

print(f"Total Games: {len(game_data)}")
print(f"Total Spreads: {len(spread_data)}")

Total Games: 5018
Total Spreads: 4775


In [4]:
pbp_data = pd.merge(pbp_data, game_data[['game_id','winner']], left_on="game_id", right_on="game_id")
pbp_data = pbp_data[pbp_data.game_id != 400603838]
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [5]:
pbp_data = pd.merge(pbp_data, spread_data[['game_id','spread']], left_on="game_id", right_on="game_id")
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [6]:
pbp_data['pos_team_spread'] = np.where(pbp_data.pos_team == pbp_data.home, pbp_data.spread, -1 * pbp_data.spread)
pbp_data['elapsed_share'] = (3600 - pbp_data.adj_TimeSecsRem) / 3600
pbp_data['spread_time'] = pbp_data.pos_team_spread * np.exp(-4 * pbp_data.elapsed_share)
pbp_data['is_home'] = np.where(pbp_data.pos_team == pbp_data.home, 1, 0)
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [7]:
pbp_data['win_indicator'] = pbp_data.pos_team == pbp_data.winner
pbp_data.half = pbp_data.half.astype(int)
pbp_data.loc[pbp_data.off_timeouts_rem_before.isna(), "off_timeouts_rem_before"] = 0
pbp_data.loc[pbp_data.def_timeouts_rem_before.isna(), "def_timeouts_rem_before"] = 0

# https://zacharyst.com/2019/03/06/group_by-mutate-using-pandas/
# def determine_receive_2h_ko(data):
# #     print(data.game_id.iloc[0])
#     val = data.sort_values(by="play_id")[(data.period == 3) & (~data.play_type.str.contains("Kickoff"))].pos_team
#     ret_val = None
#     if (len(val) != 0):
#         ret_val = val.iloc[0]
#     data['receive_2h_ko'] = ret_val
#     return data

# pbp_data = pbp_data.groupby('game_id').apply(determine_receive_2h_ko)
# pbp_data.loc[pbp_data.turnover_vec_lag.isna(), 'turnover_vec_lag'] = 0
pbp_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [8]:
filtered_pbp_data = pbp_data[
    (pbp_data.ep_before.notna())
    & (pbp_data.score_diff.notna())
    & (pbp_data.play_type.notna())
    & (pbp_data.win_indicator.notna())
    & (pbp_data.off_timeouts_rem_before.notna())
    & (pbp_data.def_timeouts_rem_before.notna())
    & (pbp_data.yards_to_goal.notna())
    & (pbp_data.period <= 4)
]

In [9]:
spread_nrounds = 760
spread_params = {
    "booster":"gbtree",
    "objective":"binary:logistic",
    "eval_metric": ["logloss"],
    "eta":0.02,
    "gamma":0.3445502,
    "subsample":0.7204741,
    "colsample_bytree":0.5714286,
    "max_depth":5,
    "min_child_weight":14
}
before_spread_columns = ["pos_team_receives_2H_kickoff","spread_time","TimeSecsRem","adj_TimeSecsRem","ExpScoreDiff_Time_Ratio","pos_score_diff_start","down","distance","yards_to_goal","is_home","pos_team_timeouts_rem_before","def_pos_team_timeouts_rem_before","period"]
after_spread_columns = ["pos_team_receives_2H_kickoff","spread_time_end","TimeSecsRem_end","adj_TimeSecsRem_end","ExpScoreDiff_Time_Ratio_end","pos_score_diff_start_end","down_end","distance_end","yards_to_goal_end","is_home","pos_team_timeouts","def_pos_team_timeouts","period"]

dtrain_spread = xgb.DMatrix(filtered_pbp_data[before_spread_columns], label=filtered_pbp_data.win_indicator)
dtrain_spread

In [10]:
wp_model_spread = xgb.train(spread_params, dtrain_spread, num_boost_round=spread_nrounds, verbose_eval=2)
# save for debug
wp_model_spread.dump_model('wp_spread_xgb_dump.json', with_stats=True, dump_format='json')

In [11]:
no_spread_nrounds = 65
no_spread_params = {
    "booster":"gbtree",
    "objective":"binary:logistic",
    "eval_metric": ["logloss"],
    "eta":0.2,
    "gamma":0,
    "subsample":0.8,
    "colsample_bytree":0.8,
    "max_depth":4,
    "min_child_weight":1
}

before_no_spread_columns = ["pos_team_receives_2H_kickoff","TimeSecsRem","adj_TimeSecsRem","ExpScoreDiff_Time_Ratio","pos_score_diff_start","down","distance","yards_to_goal","is_home","pos_team_timeouts_rem_before","def_pos_team_timeouts_rem_before","period"]
after_no_spread_columns = ["pos_team_receives_2H_kickoff","TimeSecsRem_end","adj_TimeSecsRem_end","ExpScoreDiff_Time_Ratio_end","pos_score_diff_start_end","down_end","distance_end","yards_to_goal_end","is_home","pos_team_timeouts","def_pos_team_timeouts","period"]

no_spread_dtrain = xgb.DMatrix(filtered_pbp_data[before_no_spread_columns], label=filtered_pbp_data.win_indicator)
no_spread_dtrain

In [12]:
no_spread_wp_model = xgb.train(no_spread_params, no_spread_dtrain, num_boost_round=no_spread_nrounds, verbose_eval=2)
# save for debug
no_spread_wp_model.dump_model('wp_xgb_dump.json', with_stats=True, dump_format='json')

In [13]:
wp_model = no_spread_wp_model
target_file_name = "no_spread_wpa"
model_columns_before = before_no_spread_columns
model_columns_after = after_no_spread_columns

print("MODEL DETAILS:")
print(f"Filename: {target_file_name}.mlmodel")
print(f"Using Spread: { 'spread_time' in model_columns_before}")
print(f"Model Columns: {model_columns_before}")


MODEL DETAILS:
Filename: no_spread_wpa.mlmodel
Using Spread: False
Model Columns: ['pos_team_receives_2H_kickoff', 'TimeSecsRem', 'adj_TimeSecsRem', 'ExpScoreDiff_Time_Ratio', 'pos_score_diff_start', 'down', 'distance', 'yards_to_goal', 'is_home', 'pos_team_timeouts_rem_before', 'def_pos_team_timeouts_rem_before', 'period']


In [14]:
# Test with all data
base_data = pbp_data.copy() #pd.read_parquet("https://raw.githubusercontent.com/saiemgilani/cfbscrapR-data/master/data/parquet/pbp_players_pos_2020.parquet")
# print(base_data.columns.to_list())

# games_2020 = retrieveCfbDataFile('games', 2020)
# games_2020['game_id'] = games_2020['ï»¿id']
# games_2020['year'] = 2020
# games_2020 = games_2020[games_2020.home_points != games_2020.away_points]
# games_2020['winner'] = np.where(games_2020.home_points > games_2020.away_points, games_2020.home_team, games_2020.away_team)

# spread_2020 = retrieveCfbDataFile('lines', 2020)
# spread_2020 = spread_2020[spread_2020.lineProvider == "consensus"]
# spread_2020['spread'] = -1 * spread_2020.spread # spread is negative for home favorite; positive for away favorite, but NFLfastR uses "(how many points home team was favored by) for this"
# spread_2020['game_id'] = spread_2020['ï»¿id']
# spread_2020['year'] = 2020
# base_data = pd.merge(base_data, spread_2020[['game_id','spread']], left_on="game_id", right_on="game_id")
# base_data = pd.merge(base_data, games_2020[['game_id','winner']], left_on="game_id", right_on="game_id")
base_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [15]:
base_data[base_data.down_end.isna()].play_type.value_counts()

End Period                       10801
End of Half                       4716
End of Game                       3119
Rush                              1158
Pass Incompletion                  378
Pass Reception                     313
Rushing Touchdown                  158
Timeout                            156
Passing Touchdown                  155
Penalty                            127
Field Goal Good                    121
Sack                                55
Field Goal Missed                   46
Interception Return                 34
Fumble Recovery (Own)               16
Fumble Recovery (Opponent)          14
Blocked Field Goal                   5
Uncategorized                        3
Defensive 2pt Conversion             2
Fumble Return Touchdown              2
Two Point Rush                       1
Coin Toss                            1
Interception Return Touchdown        1
Name: play_type, dtype: int64

In [16]:
base_data['win_indicator'] = np.where(base_data.pos_team == base_data.winner, 1, 0)
base_data['ExpScoreDiff_end'] = base_data.score_diff_start + base_data.ep_after
base_data['adj_TimeSecsRem_end'] = np.where(base_data.half == 1, 1800 + base_data.adj_TimeSecsRem, base_data.adj_TimeSecsRem)
base_data['ExpScoreDiff_Time_Ratio_end'] = base_data.ExpScoreDiff_end / (base_data.adj_TimeSecsRem_end + 1)
base_data['pos_team_spread'] = np.where(base_data.pos_team == base_data.home, base_data.spread, -1 * base_data.spread)
base_data['elapsed_share'] = (3600 - base_data.adj_TimeSecsRem) / 3600
base_data['elapsed_share_end'] = (3600 - base_data.adj_TimeSecsRem_end) / 3600
base_data['spread_time'] = base_data.pos_team_spread * np.exp(-4 * base_data.elapsed_share)
base_data['spread_time_end'] = base_data.pos_team_spread * np.exp(-4 * base_data.elapsed_share_end)
base_data['is_home'] = np.where(base_data.pos_team == base_data.home, 1, 0)
base_data.head()


,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [17]:
base_data = base_data[
    (base_data.ep_before.notna())
    & (base_data.pos_score_diff_start.notna())
    & (base_data.play_type.notna())
    & (base_data.win_indicator.notna())
    & (base_data.pos_team_timeouts_rem_before.notna())
    & (base_data.def_pos_team_timeouts_rem_before.notna())
    & (base_data.yards_to_goal.notna())
    & (base_data.period <= 4)

    & (base_data.ep_after.notna())
    & (base_data.pos_score_diff_start_end.notna())
    & (base_data.pos_team_timeouts.notna())
    & (base_data.def_pos_team_timeouts.notna())
    & (base_data.yards_to_goal_end.notna())

    & (base_data.down_end.notna())
]
base_data.down_end = base_data.down_end.astype(int)

In [18]:
# leave-one-season-out
def calculate_wpa(yr):
    data = base_data[base_data.year != yr]
    dtest_start = xgb.DMatrix(data[model_columns_before], label=data.win_indicator)
    WP_start = wp_model.predict(dtest_start)

    end_data = data[model_columns_after]
    end_data.columns = model_columns_before

    dtest_end = xgb.DMatrix(end_data, label=data.win_indicator)
    WP_end = wp_model.predict(dtest_end)

    base_data.loc[data.index, 'WP_NFL_start'] = WP_start
    base_data.loc[data.index,'WP_NFL_end'] = WP_end
    base_data.loc[data.index,'WPA_NFL'] = base_data['WP_NFL_end'] - base_data['WP_NFL_start']

years_range = range(min(pbp_data.year), max(pbp_data.year) + 1)
for yr in years_range:
    calculate_wpa(yr)

base_data.head()

,EPA,ExpScoreDiff,ExpScoreDiff_Time_Ratio,FG_after,FG_before,Goal_To_Go,Goal_To_Go_end,No_Score_after,No_Score_before,Opp_FG_after,Opp_FG_before,Opp_Safety_after,Opp_Safety_before,Opp_TD_after,Opp_TD_before,Safety_after,Safety_before,TD_after,TD_before,TimeSecsRem,TimeSecsRem_end,Under_three,Under_two,Under_two_end,adj_TimeSecsRem,away,away_EPA,away_EPA_pass,away_EPA_rush,away_wp_after,away_wp_before,change_of_pos_team,change_of_poss,clock.minutes,clock.seconds,completion,completion_player,completion_player_id,completion_yds,def_EPA,def_pos_team,def_pos_team_score,def_pos_team_timeouts,def_pos_team_timeouts_rem_before,def_timeout_called,def_timeouts_rem_before,def_wp_after,def_wp_before,defense_conference,defense_play,defense_score,defense_score_play,defense_timeouts,distance,distance_end,down,down_end,downs_turnover,drive_end_period,drive_end_yards_to_goal,drive_event_number,drive_id,drive_num,drive_number,drive_numbers,drive_play_number,drive_pts,drive_result,drive_result2,drive_result_detailed,drive_result_detailed_flag,drive_scoring,drive_start_period,drive_start_yards_to_goal,drive_time_minutes_elapsed,drive_time_minutes_end,drive_time_minutes_start,drive_time_seconds_elapsed,drive_time_seconds_end,drive_time_seconds_start,drive_yards,end_of_half,ep_after,ep_before,epa_success,fg_block_player_name,fg_inds,fg_kicker_player_name,fg_made,fg_make_prob,fg_return_player_name,firstD_by_kickoff,firstD_by_penalty,firstD_by_poss,firstD_by_yards,first_by_penalty,first_by_yards,fumble_forced_player,fumble_forced_player_id,fumble_forced_player_name,fumble_forced_stat,fumble_player,fumble_player_id,fumble_player_name,fumble_recovered_player,fumble_recovered_player_id,fumble_recovered_player_name,fumble_recovered_stat,fumble_stat,fumble_vec,game_id,game_play_number,half,half_clock.minutes,half_play_number,home,home_EPA,home_EPA_pass,home_EPA_rush,home_wp_after,home_wp_before,id_drive,id_play,incompletion_player,incompletion_player_id,incompletion_stat,int,int_td,interception_player,interception_player_id,interception_player_name,interception_stat,interception_thrown_player,interception_thrown_player_id,interception_thrown_stat,kick_play,kickoff_downed,kickoff_fair_catch,kickoff_onside,kickoff_oob,kickoff_play,kickoff_player_name,kickoff_returner_player_name,kickoff_safety,kickoff_tb,lag_change_of_pos_team,lag_change_of_pos_team2,lag_change_of_poss,lag_defense_score_play,lag_downs_turnover,lag_drive_result_detailed,lag_first_by_penalty,lag_first_by_penalty2,lag_first_by_yards,lag_first_by_yards2,lag_kickoff_play,lag_new_drive_pts,lag_play_text,lag_play_text2,lag_play_type,lag_play_type2,lag_play_type3,lag_pos_team,lag_punt,lag_punt2,lag_scoring_play,lag_turnover_vec,lead_drive_result_detailed,lead_play_text,lead_play_type,lead_pos_team,log_ydstogo,log_ydstogo_end,middle_8,missing_yard_flag,net_away_EPA,net_away_EPA_pass,net_away_EPA_rush,net_home_EPA,net_home_EPA_pass,net_home_EPA_rush,new_drive_pts,new_id,new_series,number_of_drives,off_timeout_called,off_timeouts_rem_before,offense_conference,offense_play,offense_receives_2H_kickoff,offense_score,offense_score_play,offense_timeouts,opponent,opponent_score,orig_drive_number,orig_play_type,pass,pass_attempt,pass_breakup_player,pass_breakup_player_id,pass_breakup_player_name,pass_breakup_stat,pass_td,passer_player_name,penalty_1st_conv,penalty_declined,penalty_detail,penalty_flag,penalty_no_play,penalty_offset,penalty_play_text,penalty_safety,penalty_text,period,play_after_turnover,play_id,play_text,play_type,pos_score_diff,pos_score_diff_start,pos_score_diff_start_end,pos_score_pts,pos_team,pos_team_receives_2H_kickoff,pos_team_score,pos_team_timeouts,pos_team_timeouts_rem_before,position_completion,position_fumble,position_fumble_forced,position_fumble_recovered,position_incompletion,position_interception,position_interception_thrown,position_pass_breakup,position_reception,position_rush,position_sack,position_sack_taken,position_target,position_touchdown,ppa,pts_scored,punt,punt_block_

In [19]:
print("Diff from cfbscrapR model:")
print(f"Diff of WPA: {(base_data['wpa'] - base_data['WPA_NFL']).mean()}")
print(f"Diff of WP_start: {(base_data['wp_before'] - base_data['WP_NFL_start']).mean()}")
print(f"Diff of WP_end: {(base_data['wp_after'] - base_data['WP_NFL_end']).mean()}")

Diff from cfbscrapR model:
Diff of WPA: -0.000252717050250481
Diff of WP_start: -0.0015268894708893753
Diff of WP_end: -0.0018431567746676441


In [20]:
base_data[["wp_before","wp_after","wpa","WP_NFL_start","WP_NFL_end","WPA_NFL"]].head()

,wp_before,wp_after,wpa,WP_NFL_start,WP_NFL_end,WPA_NFL
0,0.490320,0.471664,-0.018655,0.596446,0.561089,-0.035356
1,0.471664,0.464996,-0.006668,0.583399,0.549531,-0.033869
2,0.464996,0.554595,0.089599,0.549531,0.620825,0.071294
3,0.554595,0.521438,-0.033156,0.646667,0.608917,-0.037750
4,0.521438,0.476803,-0.044636,0.607931,0.548723,-0.059208


In [21]:
gathered = base_data.melt(id_vars=["period","win_indicator"], value_vars=["WP_NFL_start"],value_name="pred_prob")
gathered

,period,win_indicator,variable,pred_prob
0,1,1,WP_NFL_start,0.596446
1,1,1,WP_NFL_start,0.583399
2,1,1,WP_NFL_start,0.549531
3,1,1,WP_NFL_start,0.646667
4,1,1,WP_NFL_start,0.607931
...,...,...,...,...
845808,4,0,WP_NFL_start,0.006894
845809,4,1,WP_NFL_start,0.973674
845810,4,1,WP_NFL_start,0.972813
845811,4,1,WP_NFL_start,0.975368


In [22]:
gathered['bin_pred_prob'] = (round(gathered.pred_prob / 0.05) * .05)
gathered

,period,win_indicator,variable,pred_prob,bin_pred_prob
0,1,1,WP_NFL_start,0.596446,0.60
1,1,1,WP_NFL_start,0.583399,0.60
2,1,1,WP_NFL_start,0.549531,0.55
3,1,1,WP_NFL_start,0.646667,0.65
4,1,1,WP_NFL_start,0.607931,0.60
...,...,...,...,...,...
845808,4,0,WP_NFL_start,0.006894,0.00
845809,4,1,WP_NFL_start,0.973674,0.95
845810,4,1,WP_NFL_start,0.972813,0.95
845811,4,1,WP_NFL_start,0.975368,1.00


In [23]:
gathered['bin_pred_prob'].value_counts()

0.00    102873
1.00     98172
0.95     42588
0.05     40889
0.70     40078
0.90     38002
0.10     36327
0.65     36059
0.15     35439
0.20     35257
0.35     34007
0.60     33530
0.85     32434
0.40     32249
0.45     31734
0.75     31068
0.50     30085
0.25     29814
0.55     29799
0.30     29519
0.80     25890
Name: bin_pred_prob, dtype: int64

In [24]:
grouped_score = gathered.groupby(['period','bin_pred_prob'])
def organize(grp):
    d = {}
    d['n_plays'] = len(grp)
    d['n_wins'] = len(grp[grp.win_indicator == True])
    d['bin_actual_prob'] = d['n_wins'] / d['n_plays']
    return pd.Series(d, index=['n_plays','n_wins','bin_actual_prob'])

organized = grouped_score.apply(organize)
organized = organized.reset_index()
organized

,period,bin_pred_prob,n_plays,n_wins,bin_actual_prob
0,1,0.00,585.0,2.0,0.003419
1,1,0.05,3848.0,150.0,0.038981
2,1,0.10,4990.0,382.0,0.076553
3,1,0.15,7300.0,1080.0,0.147945
4,1,0.20,10706.0,2031.0,0.189707
5,1,0.25,9090.0,2215.0,0.243674
6,1,0.30,9999.0,2921.0,0.292129
7,1,0.35,15074.0,5170.0,0.342975
8,1,0.40,15875.0,6235.0,0.392756
9,1,0.45,16382.0,7419.0,0.452875


In [25]:
organized['cal_diff'] = abs(organized.bin_pred_prob - organized.bin_actual_prob)
organized

,period,bin_pred_prob,n_plays,n_wins,bin_actual_prob,cal_diff
0,1,0.00,585.0,2.0,0.003419,0.003419
1,1,0.05,3848.0,150.0,0.038981,0.011019
2,1,0.10,4990.0,382.0,0.076553,0.023447
3,1,0.15,7300.0,1080.0,0.147945,0.002055
4,1,0.20,10706.0,2031.0,0.189707,0.010293
5,1,0.25,9090.0,2215.0,0.243674,0.006326
6,1,0.30,9999.0,2921.0,0.292129,0.007871
7,1,0.35,15074.0,5170.0,0.342975,0.007025
8,1,0.40,15875.0,6235.0,0.392756,0.007244
9,1,0.45,16382.0,7419.0,0.452875,0.002875


In [26]:
def organize_cal_err(grp):
    d = {}
    d['weight_cal_err'] = np.average(grp.cal_diff, weights=grp.n_plays)
    d['n_wins'] = sum(grp.n_wins)
    return pd.Series(d, index=['weight_cal_err', 'n_wins'])

cv_cal_error = organized.groupby('period')
final_cal_error = cv_cal_error.apply(organize_cal_err)
final_cal_error = final_cal_error.reset_index()
final_cal_error

,period,weight_cal_err,n_wins
0,1,0.007508,102506.0
1,2,0.005421,115367.0
2,3,0.004216,99577.0
3,4,0.007235,103408.0


In [27]:
weighted_cal_err_final = np.average(final_cal_error.weight_cal_err, weights=final_cal_error.n_wins)
print(f"Weighted calibration error: {weighted_cal_err_final}")

Weighted calibration error: 0.006089925225397449


In [28]:
import sys
!{sys.executable} -m pip install coremltools==3.4

In [31]:
# # CoreML model
import coremltools as cml
# print(cml.version)

cml_inputs = {
    "pos_team_receives_2H_kickoff" : "If the current offense will receive the second half kickoff",
#     "spread_time" : "The number of points that the offense is favored by, scaled by e^x, where x is the elapsed portion of the game ((3600 - adj_TimeSecsRem) / 3600)",
    "TimeSecsRem" : "The number of seconds left in the half.",
    "adj_TimeSecsRem" : "The number of seconds left in the game.",
    "ExpScoreDiff_Time_Ratio" : "The expected point differential divided by the number of seconds remaining in the game (plus 1).",
    "pos_score_diff_start" : "The current difference in score from the offense's perspective.",
    "down" : "The current down.",
    "distance" : "How many yards the offense must gain for a first down.",
    "yards_to_goal" : "The yards left until the current offense reaches the goal line.",
    "is_home" : "If the current offense is the home team",
    "pos_team_timeouts_rem_before" : "How many timeouts the current offense has remaining",
    "def_pos_team_timeouts_rem_before" : "How many timeouts the current defense has remaining",
    "period" : "The current quarter."
}

target_outputs = {
    "column" : "win_indicator",
    "description" : "The probability of the current offense winning the game."
}

def create_coreml_model(xgb_model, author, description, input_details, target_details, file_name):
    cml_model = cml.converters.xgboost.convert(xgb_model, mode="regressor", target=target_details['column'])

    cml_model.author = author#'Ben Baldwin (@benbbaldwin on Twitter, @guga31bb on GitHub) and Sebastian Carl (@mrcaseb on Twitter/GitHub) for model creation code; Saiem Gilani and Meyappan Subbiah for cfbscrapR training data; Akshay Easwaran for Python port.'
    cml_model.license = 'MIT'
    cml_model.short_description = description #''

    # Set feature descriptions manually
    for (column, desc) in input_details.items():
        cml_model.input_description[column] = desc

    # Set the output descriptions
    cml_model.output_description[target_details['column']] = target_details['description']

    # Save the model
    cml_model.save(f'{file_name}.mlmodel')
#     cml_model.get_spec()
    return cml_model
    
coreml_mdl_file = create_coreml_model(wp_model, "Ben Baldwin (@benbbaldwin on Twitter, @guga31bb on GitHub) and Sebastian Carl (@mrcaseb on Twitter/GitHub) for model creation code; Saiem Gilani and Meyappan Subbiah for cfbscrapR training data; Akshay Easwaran for Python port.", "Predicts the win probability of the offense for a given play. Translated from R, original model available as part of https://github.com/mrcaseb/nflfastR/.", cml_inputs, target_outputs, target_file_name)
coreml_mdl_file.get_spec()

specificationVersion: 1
description {
  input {
    name: "pos_team_receives_2H_kickoff"
    shortDescription: "If the current offense will receive the second half kickoff"
    type {
      doubleType {
      }
    }
  }
  input {
    name: "TimeSecsRem"
    shortDescription: "The number of seconds left in the half."
    type {
      doubleType {
      }
    }
  }
  input {
    name: "adj_TimeSecsRem"
    shortDescription: "The number of seconds left in the game."
    type {
      doubleType {
      }
    }
  }
  input {
    name: "ExpScoreDiff_Time_Ratio"
    shortDescription: "The expected point differential divided by the number of seconds remaining in the game (plus 1)."
    type {
      doubleType {
      }
    }
  }
  input {
    name: "pos_score_diff_start"
    shortDescription: "The current difference in score from the offense\'s perspective."
    type {
      doubleType {
      }
    }
  }
  input {
    name: "down"
    shortDescription: "The current down."
    type {
      do

In [ ]:
test_json = pd.read_json("wp_xgb_dump.json")
test_json.head()

In [ ]:
len(test_json[test_json['split_condition'].isna()])